In [1]:


import sys
sys.path.append('~/msmsgym/msnovelist-private')

import infrastructure.preprocessing as pp
import sqlite3
import yaml
import fp_management.fp_database as db
import fp_management.fp_database_sqlite
from tqdm import tqdm
from iteration_utilities import duplicates


PROCESSING_BLOCK_MAX_COUNT=9999999999
#PROCESSING_BLOCK_MAX_COUNT=100000


config_ = "/home/stravsmi/msmsgym/MSNovelist-private/preprocessing_mist/log.yaml"
with open(config_, 'r') as f:
    config = yaml.safe_load(f)

db_train = config["db_step1"]


In [2]:
con = sqlite3.connect(db_train)
with con:
    cur = con.cursor()

    q = "CREATE INDEX IF NOT EXISTS inchikeys ON compounds (inchikey1)"
    res = cur.execute(q)

    q = "SELECT inchikey1 FROM compounds"
    res = cur.execute(q)
    keys = res.fetchall()
    keys_keys = [x[0] for x in keys]
    keys_set = set(keys_keys)

    dups = list(duplicates(keys_keys))
    dups_killed = {}
    q = "UPDATE compounds SET grp = 'invalid' WHERE inchikey1 = ?"
    for dup in tqdm(dups):
        res = cur.execute(q, [dup])
        cnt = cur.rowcount
        dups_killed[dup] = cnt



100%|██████████| 11/11 [00:00<00:00, 15057.88it/s]


In [3]:
con.close()